In [98]:
import datetime
import pandas as pd
import numpy as np
import seaborn as sns

First we'll pull in the raw Discourse (forum) usage data and convert it to a pandas df.

The Discourse data is pulled via SQL using the Discourse Data Explorer plugin. Exported data is
limited to 10,000 rows per query, so we'll pull in each csv and concatenate them.

In [99]:
concat_list = []
for i in range(1,7):
    df = pd.read_csv(f"raw-interval-user-stats-{i}.csv")
    df["visited_at"] = pd.to_datetime(df["visited_at"])
    concat_list.append(df)

discourse_df = pd.concat(concat_list, ignore_index=True, sort=False).drop_duplicates()

Next, we'll do the same for the raw Skedda (equipment usage) data.

Skedda allows data exports to CSVs for intervals up to 1 year, so our final dataframe will 
concatenate all of the available years.

In [100]:
skedda_ranges = [
    "feb2020_to_feb2021",
    "feb2021_to_feb2022",
    "feb2022_to_feb2023",
    "feb2023_to_feb2024",
    "feb2024_to_feb2025",
]
skedda_concats = []
for date_range in skedda_ranges:
    df = pd.read_csv(f"skedda_bookings_{date_range}.csv")
    df["Scheduled start"] = pd.to_datetime(df["Scheduled start"])
    df["End"] = pd.to_datetime(df["End"])
    skedda_concats.append(df)

skedda_df = pd.concat(skedda_concats, ignore_index=True, sort=False).drop_duplicates()

Next, we'll pull in the Neon member data we've collected via get_asmbly_member_data_long_form.py

For each user, we'll find the Discourse usage stats and Skedda usage stats for each period of
membership (if any). We'll also calculate the change in usage from period-to-period.

In [101]:
df = pd.read_csv("all_members_long_form.csv")

base_discourse_cols = ["discourse_read_time", "discourse_posts_read", "discourse_posts_made"]

user_df_dict = {}
for row in df.itertuples():
    if (user_df := user_df_dict.get(row.discourse_id)) is None:
        user_df = discourse_df[discourse_df.username == row.discourse_id]
        user_df_dict[row.discourse_id] = user_df

    time_restricted_df = user_df[(user_df.visited_at <= row.end_date) & (user_df.visited_at >= row.start_date)]

    for col in base_discourse_cols:
        split = col.split("_")
        new = split[2] + "_" + split[1] if "time" in split else split[1] + "_" + split[2]
        df.loc[row.Index, col] = time_restricted_df[new].sum()

    try:
        prev_id = df.loc[row.Index-1, "discourse_id"]
    except KeyError:
        for col in base_discourse_cols:
            df.loc[row.Index, f"change_{col}"] = df.loc[row.Index, col]
        continue

    for col in base_discourse_cols:
        df.loc[row.Index, f"change_{col}"] = (df.loc[row.Index, col] - df.loc[(row.Index - 1), col]) if prev_id == row.discourse_id else df.loc[row.Index, col]


In [102]:
user_df_dict = {}
for row in df.itertuples():
    if (user_df := user_df_dict.get(row.email)) is None:
        user_df = skedda_df[skedda_df["Holder email"] == row.email]
        user_df_dict[row.email] = user_df

    time_restricted_df = user_df[(user_df["Scheduled start"] <= row.end_date) & (user_df["Scheduled start"] >= row.start_date)]

    df.loc[row.Index, "skedda_booking_count"] = time_restricted_df["Duration (minutes)"].count()
    df.loc[row.Index, "skedda_booking_minutes"] = time_restricted_df["Duration (minutes)"].sum()

    try:
        prev_email = df.loc[row.Index-1, "email"]
    except KeyError:
        df.loc[row.Index, "change_skedda_booking_count"] = df.loc[row.Index, "skedda_booking_count"]
        df.loc[row.Index, "change_skedda_booking_minutes"] = df.loc[row.Index, "skedda_booking_minutes"]
        continue

    df.loc[row.Index, "change_skedda_booking_count"] = (df.loc[row.Index, "skedda_booking_count"] - df.loc[(row.Index - 1), "skedda_booking_count"]) if prev_email == row.email else df.loc[row.Index, "skedda_booking_count"]
    df.loc[row.Index, "change_skedda_booking_minutes"] = (df.loc[row.Index, "skedda_booking_minutes"] - df.loc[(row.Index - 1), "skedda_booking_minutes"]) if prev_email == row.email else df.loc[row.Index, "skedda_booking_minutes"]


Finally, one additional variable type that could be relevant in our time-varying survival analysis is the 
cumulative sum of time-varying variables. This is in essence a measure of the "sunk cost" for the member over time.

In [103]:
time_var_cols = [
    "discourse_read_time",
    "discourse_posts_made",
    "discourse_posts_read",
    "skedda_booking_count",
    "skedda_booking_minutes",
    "num_classes_attended",
    "dollars_spent",
    "woodshop_classes",
    "metal_shop_classes",
    "electronics_classes",
    "textiles_classes",
    "lasers_classes",
    "3dp_classes"
]

for row in df.itertuples():

    try:
        prev_id = df.loc[row.Index-1, "neon_id"]
    except KeyError:
        for col in time_var_cols:
            df.loc[row.Index, f"cum_{col}"] = df.loc[row.Index, col]
        continue

    for col in time_var_cols:
        df.loc[row.Index, f"cum_{col}"] = (df.loc[row.Index, col] + df.loc[(row.Index - 1), f"cum_{col}"]) if prev_id == row.neon_id else df.loc[row.Index, col]

Now let's deal with the remaining missing values in the age, gender, distance and time from Asmbly,
and referral source columns. Depending on the model used, the model itself may be able to handle 
the missing data without issue (i.e. ensemble tree models). However, for now we'll assume we're 
going to use a time-varying Cox Proportional Hazards model, where we need to handle missing values.

Accounts without a referral source were all created before we began tracking this metric. We can 
assume that these old accounts found out about Asmbly through their own searching (i.e. Google), 
since Google was one of the only means of finding Asmbly back then. This also happens to coincide
with the mode of referral source.

There are a relatively small number of accounts missing distance/time data due to malformed addresses. 
For these, we'll just use the median values.

For age and gender, we could use the FiveThirtyEight approach of imputation based on name, but there
are a number of issues with prediciting age and a binary gender based on name alone. Because of this, 
we will simply use the median and mode, respectively.

In [104]:
distance_med = df["distance_from_asmbly"].median()
time_med = df["time_from_asmbly"].median()

referral_mode = df["referral_source"].mode()[0]

age_med = df["age"].median()
gender_mode = df["gender"].mode()[0]

df["distance_from_asmbly"].fillna(distance_med, inplace=True)
df["time_from_asmbly"].fillna(time_med, inplace=True)
df["referral_source"].fillna(referral_mode, inplace=True)
df["age"].fillna(age_med, inplace=True)
df["gender"].fillna(gender_mode, inplace=True)

We no longer need a few of these columns now that we've joined all the data, so let's drop them.

We'll also rename a few of the columns to be a bit more understandable.

In [105]:
columns_to_drop = [
    "email",
    "start_date",
    "end_date",
    "discourse_id",
    "first_name",
    "last_name",
]

df.drop(columns=columns_to_drop, inplace=True)

rename = {
    "woodshop_safety": "taken_WSS_class",
    "lasers": "taken_lasers_class",
    "cnc_router": "taken_cnc_class",
    "num_classes_attended": "num_classes_attended_last_period",
    "metal_shop_safety": "taken_MSS_class",
}

df.rename(columns=rename, inplace=True)

Finally, we'll convert out annual membership start and stop periods to the same unit (months) as 
the monthly memberships.

In [106]:
df.loc[df.membership_type == "YEAR", ["start", "stop"]] *= 12

In [107]:
df.to_csv("all_members_long_form_cleaned.csv", index=False)

In [108]:
len(df["neon_id"].unique())

1247